In [5]:
import requests
import time

# Initialization 
API_KEY = {'X-API-key': 'HCA61YVQ'}
s = requests.Session() 
s.headers.update(API_KEY)

resp = s.get('http://localhost:9999/v1/case')  # removed space before get
resp  # 200 means success

<Response [200]>

In [41]:
def get_tick():
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        return resp.json()['tick']
    return 0

def get_position(ticker):
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        for sec in resp.json():
            if sec['ticker'] == ticker:
                return sec['position']
    return 0

def send_order(ticker, action, price, qty):
    params = {
        'ticker': ticker,
        'type': 'LIMIT',
        'quantity': qty,
        'action': action,
        'price': price
    }
    resp = s.post('http://localhost:9999/v1/orders', params=params)
    if resp.ok:
        print(f'✓ {action} {qty} @ {price}')
        return True
    else:
        print(f'✗ FAILED: {resp.text}')
        return False

def cancel_all():
    s.post('http://localhost:9999/v1/commands/cancel', params={'all': 1})
    time.sleep(0.3)

def get_market_info_filtered(ticker):
    """Get best bid/ask EXCLUDING our team (9A, 9B, 9C)"""
    resp = s.get('http://localhost:9999/v1/securities/book', 
                 params={'ticker': ticker})
    if resp.ok:
        book = resp.json()
        
        # Our team members to filter out
        teammates = ['Team 9A', 'Team 9B', 'Team 9C']
        
        # Find best bid NOT from our team
        best_bid = None
        for bid in book['bids']:
            trader = bid.get('trader_id', '')
            if trader not in teammates:
                best_bid = bid['price']
                break
        
        # Find best ask NOT from our team
        best_ask = None
        for ask in book['asks']:
            trader = ask.get('trader_id', '')
            if trader not in teammates:
                best_ask = ask['price']
                break
        
        if best_bid and best_ask:
            mid = (best_bid + best_ask) / 2
        else:
            mid = 500
        
        return best_bid, best_ask, mid
    
    return None, None, 500

def get_my_orders():
    resp = s.get('http://localhost:9999/v1/orders', params={'ticker': 'APPL'})
    if resp.ok:
        return resp.json()
    return []

def get_news():
    resp = s.get('http://localhost:9999/v1/news')
    if resp.ok:
        return resp.json()
    return []

In [ ]:

# ============================================
# MAIN STRATEGY
# ============================================

print('='*60)
print('APPL DUAL STRATEGY - LIVE v2')
print('='*60)

harvest_traded = False
my_harvest = None
last_tick = 0

while True:
    tick = get_tick()
    
    # Stop conditions
    if tick <= 5:
        time.sleep(1)
        continue
    if tick > 895:
        print('\nCase ending...')
        break
    
    pos = get_position('APPL')
    
    # ==================================================
    # STRATEGY 2: HARVEST BOMB (Tick 180+) - CHECK FIRST
    # ==================================================
    if tick >= 180 and not harvest_traded:
        print('\n' + '='*60)
        print('TICK 180 - HARVEST NEWS!')
        print('='*60)
        
        time.sleep(0.5)
        news = get_news()
        
        for item in news:
            headline = item.get('headline', '')
            if 'personal harvest of apples' in headline.lower():
                words = headline.split()
                for i, word in enumerate(words):
                    if word == 'is' and i + 1 < len(words):
                        try:
                            my_harvest = int(words[i + 1].replace('.', ''))
                            break
                        except:
                            pass
        
        if my_harvest:
            print(f'\n🍎 YOUR HARVEST: {my_harvest} 🍎\n')
            
            market_mid = get_market_info_filtered('APPL')[2]
            
            # Risk-managed sizing
            if my_harvest >= 80:
                distance = my_harvest - 80
                confidence = min(distance / 10, 1.0)
                size = int(300 + 200 * confidence)
                price = market_mid + 20
                
                cancel_all()
                send_order('APPL', 'BUY', price, size)
                print(f'🚀 HIGH HARVEST: Buying {size} @ {price}')
                print(f'   Confidence: {confidence*100:.0f}%\n')
                
            elif my_harvest <= 20:
                distance = 20 - my_harvest
                confidence = min(distance / 10, 1.0)
                size = int(300 + 200 * confidence)
                price = market_mid - 20
                
                cancel_all()
                send_order('APPL', 'SELL', price, size)
                print(f'🚀 LOW HARVEST: Selling {size} @ {price}')
                print(f'   Confidence: {confidence*100:.0f}%\n')
                
            else:
                if my_harvest > 50:
                    size = 100
                    price = market_mid + 10
                    cancel_all()
                    send_order('APPL', 'BUY', price, size)
                    print(f'📊 MODERATE: Small buy {size} @ {price}\n')
                else:
                    size = 100
                    price = market_mid - 10
                    cancel_all()
                    send_order('APPL', 'SELL', price, size)
                    print(f'📊 MODERATE: Small sell {size} @ {price}\n')
            
            harvest_traded = True
            time.sleep(2)
            print(f'Position after harvest trade: {get_position("APPL")}\n')
            print('='*60)
    
    # ==================================================
    # STRATEGY 1: SMART BOT SCALPER (Continuous)
    # ==================================================
    if tick != last_tick:
        # FILTER OUT TEAMMATES!
        best_bid, best_ask, mid = get_market_info_filtered('APPL')
        my_orders = get_my_orders()
        
        # Check our current orders
        my_best_bid = None
        my_best_ask = None
        for order in my_orders:
            if order['action'] == 'BUY' and order['status'] == 'OPEN':
                my_best_bid = order['price']
            if order['action'] == 'SELL' and order['status'] == 'OPEN':
                my_best_ask = order['price']
        
        # ==================================================
        # POSITION LIMITS AFTER HARVEST
        # ==================================================
        skip_buys = False
        skip_sells = False
        
        if harvest_traded and my_harvest:
            if my_harvest >= 80:  # High harvest - should be LONG
                if pos < -200:  # Too short!
                    print(f'⚠️ Pos {pos} too short for harvest {my_harvest} - ONLY BUYING')
                    skip_sells = True
                elif pos < 0:  # Still short but less extreme
                    skip_sells = True  # Don't sell more
                    
            elif my_harvest <= 20:  # Low harvest - should be SHORT
                if pos > 200:  # Too long!
                    print(f'⚠️ Pos {pos} too long for harvest {my_harvest} - ONLY SELLING')
                    skip_buys = True
                elif pos > 0:  # Still long but less extreme
                    skip_buys = True  # Don't buy more
        
        # ==================================================
        # DIRECTIONAL BIAS AFTER HARVEST
        # ==================================================
        if best_bid and best_ask:
            if harvest_traded and my_harvest:
                if my_harvest >= 80:
                    # High harvest - want to be long, widen offers
                    my_bid = best_bid + 0.01  # Aggressive bids
                    my_ask = best_ask + 50    # Wide offers (don't want to sell)
                    
                elif my_harvest <= 20:
                    # Low harvest - want to be short, widen bids
                    my_bid = best_bid - 50    # Wide bids (don't want to buy)
                    my_ask = best_ask - 0.01  # Aggressive offers
                    
                else:
                    # Moderate - normal market making
                    my_bid = best_bid
                    my_ask = best_ask
            else:
                # Pre-harvest - normal tight market making
                spread = best_ask - best_bid
                if spread > 5:
                    my_bid = best_bid + 0.01
                    my_ask = best_ask - 0.01
                else:
                    my_bid = best_bid
                    my_ask = best_ask
        else:
            # No market from non-teammates - post wide
            my_bid = mid - 20
            my_ask = mid + 20
        
        # ==================================================
        # UPDATE ORDERS (WITH POSITION LIMITS)
        # ==================================================
        need_update = False
        
        if my_best_bid is None or my_best_bid < my_bid:
            need_update = True
        elif my_best_bid:
            my_bid = my_best_bid
        
        if my_best_ask is None or my_best_ask > my_ask:
            need_update = True
        elif my_best_ask:
            my_ask = my_best_ask
        
        if need_update or len(my_orders) == 0:
            cancel_all()
            
            # Only post orders if not blocked by position limits
            if not skip_buys:
                send_order('APPL', 'BUY', my_bid, 200)
            else:
                print(f'Skipping buy - position limit')
                
            if not skip_sells:
                send_order('APPL', 'SELL', my_ask, 200)
            else:
                print(f'Skipping sell - position limit')
        
        print(f'Tick {tick} | Pos: {pos} | Bid: {my_bid} | Ask: {my_ask}')
        last_tick = tick
    
    time.sleep(0.5)

# Final cleanup
cancel_all()
final_pos = get_position('APPL')
print(f'\n{"="*60}')
print(f'FINAL POSITION: {final_pos}')
print('DONE!')
print('='*60)


APPL DUAL STRATEGY - LIVE v2

TICK 180 - HARVEST NEWS!

🍎 YOUR HARVEST: 27 🍎

✓ SELL 100 @ 490
📊 MODERATE: Small sell 100 @ 490

Position after harvest trade: 0.0

✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 409 | Pos: 0.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 413 | Pos: 200.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 414 | Pos: 400.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 415 | Pos: 600.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 416 | Pos: 800.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 417 | Pos: 1000.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 418 | Pos: 1200.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 419 | Pos: 1400.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 420 | Pos: 1600.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 200 @ 490.0
Tick 421 | Pos: 1800.0 | Bid: 480 | Ask: 490.0
✓ BUY 200 @ 480
✓ SELL 